In [4]:
import pandas as pd
import numpy as np
from itertools import product
import math
import random
import copy

According to Hinton's "A Practical Guide to Training Restricted Boltzmann Machines"

Initialize J matrix to zero mean and 0.01 sd. The diagonal will be 0.

Initizlize bias b_i  of visible unit i to log[pi/(1−pi)]

In [194]:
class IsingModel:
    def __init__(self, data, lr=0.1):
        self.lr = lr
        
        # Possible values the spins can take
        self.spin_values = [-1,1]
        
        self.data_activations_matrix = pd.DataFrame(activations_list, columns=['sigma_0', 'sigma_1', 'sigma_2', 'sigma_3'])
        # Calculate the true data expectations
        self.data_spins_correlation = self.data_activations_matrix.corr().values
        lind = np.tril_indices_from(self.data_spins_correlation)
        self.data_spins_correlation[lind]=0
        self.data_spins_average = self.data_activations_matrix.mean().values
        
        self.num_spins = self.data_activations_matrix.shape[1]
        self.num_samples_data = self.data_activations_matrix.shape[0]

        # Calculate the initial J matrix. Fill only the upper diagonal
        # Filling with Normal(0, 0.01)
        np.random.seed(3)
        self.J = np.random.normal(loc=0.0, scale=.01, size=self.data_spins_correlation.shape)
        lind = np.tril_indices_from(self.J)
        self.J[lind]=0
                
        # Calculate the initial H vector, each entry proportional to its probability (according to Hinton)
        self.H = self.data_activations_matrix.sum()/self.num_samples_data
        # Try with random H
        #self.H = np.random.normal(loc=0.0, scale=.01, size=self.num_spins)
    
    # Calcualte the energy of the model's parameters given a sigma state
    def calculate_energy(self, sigma):
        energy = 0
        for i in range(0, self.num_spins):
            for j in range(i+1, self.num_spins):
                energy += sigma[i]*sigma[j]*self.J[i][j]

        for i in range(0, self.num_spins):
            energy += sigma[i]*self.H[i]

        return - energy
    
    # Flip a randomly selected spin
    def flip_spin(self, _sigma):
        sigma = copy.deepcopy(_sigma)
        spin_to_flip = random.choice(range(0, self.num_spins))

        spin = sigma[spin_to_flip]
        if spin == self.spin_values[0]:
            sigma[spin_to_flip] = self.spin_values[1]
        else:
            sigma[spin_to_flip] = self.spin_values[0]
        return sigma
        
        
    
    # Train the exact model
    def train(self, max_epochs = 500):
        
        totalParamVariation = math.inf
        stopCondition = 0.005
        
        epoch = 1
        
        while totalParamVariation > stopCondition and epoch < 500:
            # Calculate P(sigma) for every possible combination in the model

            prob_dict = {}
            for sigma in product(self.spin_values, repeat=self.num_spins):
                prob_dict[sigma] = math.exp(-self.calculate_energy(sigma))

            # Calculate the partition function
            Z = sum(prob_dict.values())

            # Normalize dividing by the partition function
            for sigma in prob_dict.keys():
                prob_dict[sigma] = prob_dict[sigma]/Z

            corr_model_exp = np.zeros(self.J.shape)
            mean_model_exp = np.zeros(self.H.shape)
            for sigma in product(self.spin_values, repeat=self.num_spins):
                
                # Calculate the expectation of the correlations
                for i in range(0, self.num_spins):
                    for j in range(i+1, self.num_spins):
                        corr_model_exp[i][j] += sigma[i]*sigma[j]*prob_dict[sigma]

                # Calculate the expectation of the averages
                for i in range(0, self.num_spins):
                    mean_model_exp[i] += sigma[i]*prob_dict[sigma]


            # Calculate the step size for every J_{ij}
            stepJ = self.lr * (self.data_spins_correlation - corr_model_exp)
            # Calculate the variation of J for the termination condition
            deltaJ = abs(self.J - stepJ).sum().sum()
            # Take the step
            self.J = self.J + stepJ

            # Calculate the step size for every H_{i}
            stepH = self.lr * (self.data_spins_average - mean_model_exp)
            # Calculate the variation of J for the termination condition
            deltaH = abs(self.H - stepH).sum()
            # Take the step
            self.H = self.H + stepH
            totalParamVariation = deltaH + deltaJ
            
            print('Epoch', epoch, 'TotalParamVariation', totalParamVariation)
            
            epoch += 1

        print(self.H)
        print(self.data_spins_correlation)
        print(self.J)
        
    def random_walk(self, max_steps=40):
        # Generate a random initial spin configuration
        model_activations_list = []
        sigma_0 = random.choices([-1,1], k=self.num_spins)
        
        # Calculate the energy at this time
        energy_0 = self.calculate_energy(sigma_0)
        
        for i in range(0, max_steps):
            
            # Flip a spin and calculate the new energy
            sigma_t = self.flip_spin(sigma_0)
            energy_t = self.calculate_energy(sigma_t)
            delta_energy = energy_t - energy_0

            if delta_energy<=0: # If delta_energy<0, accept the spin flip, energetically favourable
                model_activations_list.append(sigma_t)
            

            else: # Check if transition probability p < uniform r
                p = math.exp(-delta_energy)
                r = random.random()

                if r<=p: # Accept spin flip
                    model_activations_list.append(sigma_t)
                else: # Refuse spin flip
                    model_activations_list.append(sigma_0)
            
            sigma_0 = sigma_t
            energy_0 = energy_t
        
        return model_activations_list
    
    def train_metropolis(self, max_epochs = 300):
        random.seed(3)
        totalParamVariation = math.inf
        stopCondition = 0.0005
        
        epoch = 1
        randomWalkLength = 40
        
        while totalParamVariation > stopCondition and epoch < max_epochs:
            model_activations_list = self.random_walk(randomWalkLength)   
            model_activations_matrix = pd.DataFrame(model_activations_list, columns=['sigma_0', 'sigma_1', 'sigma_2', 'sigma_3'])
            # Calculate teh data expectations
            model_spins_correlation = model_activations_matrix.corr().values
            lind = np.tril_indices_from(model_spins_correlation)
            model_spins_correlation[lind]=0
            model_spins_average = model_activations_matrix.mean().values
            
            # Calculate the step size for every J_{ij}
            stepJ = self.lr * (self.data_spins_correlation - model_spins_correlation)
            # Calculate the variation of J for the termination condition
            deltaJ = abs(self.J - stepJ).sum().sum()
            # Take the step
            self.J = self.J + stepJ

            # Calculate the step size for every H_{i}
            stepH = self.lr * (self.data_spins_average - model_spins_average)
            # Calculate the variation of J for the termination condition
            deltaH = abs(self.H - stepH).sum()
            # Take the step
            self.H = self.H + stepH
            totalParamVariation = deltaH + deltaJ
            
            
            print('Epoch', epoch, 'TotalParamVariation', totalParamVariation)
            
            epoch +=1

        print(self.H)
        print(self.data_spins_correlation)
        print(self.J)
        

In [195]:
activations_list = [[1,-1,1,-1], [1,-1,1,-1], [-1,1,-1,-1], [-1,1,1,1], [1,1,1,-1], [-1,-1,1,1], [1,-1,-1,1], [1,-1,1,1]]

In [196]:
ising = IsingModel(activations_list, lr=0.0005)

In [198]:
ising.train_metropolis(max_epochs = 600)


Epoch 1 TotalParamVariation 1.0386251871067014
Epoch 2 TotalParamVariation 1.0409685621272495
Epoch 3 TotalParamVariation 1.0411801991156893
Epoch 4 TotalParamVariation 1.0421684409112768
Epoch 5 TotalParamVariation 1.042256708046085
Epoch 6 TotalParamVariation 1.0433404201006085
Epoch 7 TotalParamVariation 1.0424107375902232
Epoch 8 TotalParamVariation 1.0443897299082576
Epoch 9 TotalParamVariation 1.0434225142315678
Epoch 10 TotalParamVariation 1.0435380811498343
Epoch 11 TotalParamVariation 1.0443926240436652
Epoch 12 TotalParamVariation 1.0450715537406852
Epoch 13 TotalParamVariation 1.0452122539945772
Epoch 14 TotalParamVariation 1.0452558066241338
Epoch 15 TotalParamVariation 1.0464874071014318
Epoch 16 TotalParamVariation 1.0467225530182034
Epoch 17 TotalParamVariation 1.0480430468213973
Epoch 18 TotalParamVariation 1.048737313495888
Epoch 19 TotalParamVariation 1.0487808553572215
Epoch 20 TotalParamVariation 1.0510161251091168
Epoch 21 TotalParamVariation 1.0505111317133733
Epo

Epoch 239 TotalParamVariation 1.2609566281079856
Epoch 240 TotalParamVariation 1.2617781232535044
Epoch 241 TotalParamVariation 1.2627687579847833
Epoch 242 TotalParamVariation 1.2642370099350346
Epoch 243 TotalParamVariation 1.265117877159387
Epoch 244 TotalParamVariation 1.2650064889705646
Epoch 245 TotalParamVariation 1.2654940761698699
Epoch 246 TotalParamVariation 1.2670801262687994
Epoch 247 TotalParamVariation 1.2678627547271266
Epoch 248 TotalParamVariation 1.2692237941500748
Epoch 249 TotalParamVariation 1.2715848214802778
Epoch 250 TotalParamVariation 1.2719943954350843
Epoch 251 TotalParamVariation 1.2727102655466938
Epoch 252 TotalParamVariation 1.2730257973612211
Epoch 253 TotalParamVariation 1.2727395553309466
Epoch 254 TotalParamVariation 1.2744108827803853
Epoch 255 TotalParamVariation 1.2744993700907488
Epoch 256 TotalParamVariation 1.276992412141643
Epoch 257 TotalParamVariation 1.2778919962418893
Epoch 258 TotalParamVariation 1.278121629337511
Epoch 259 TotalParamVar

Epoch 479 TotalParamVariation 1.47171665969698
Epoch 480 TotalParamVariation 1.4732120463220644
Epoch 481 TotalParamVariation 1.473936845196124
Epoch 482 TotalParamVariation 1.474072106356728
Epoch 483 TotalParamVariation 1.4759441487935128
Epoch 484 TotalParamVariation 1.4771685333089528
Epoch 485 TotalParamVariation 1.478127205699743
Epoch 486 TotalParamVariation 1.4803198004186575
Epoch 487 TotalParamVariation 1.4808650568171797
Epoch 488 TotalParamVariation 1.4812364690962814
Epoch 489 TotalParamVariation 1.4814306407479463
Epoch 490 TotalParamVariation 1.4827289445150402
Epoch 491 TotalParamVariation 1.4838973253169272
Epoch 492 TotalParamVariation 1.4850542461734635
Epoch 493 TotalParamVariation 1.4844580796393076
Epoch 494 TotalParamVariation 1.4861909087591767
Epoch 495 TotalParamVariation 1.486343221897879
Epoch 496 TotalParamVariation 1.487244814579109
Epoch 497 TotalParamVariation 1.4886216211604526
Epoch 498 TotalParamVariation 1.4881576504814569
Epoch 499 TotalParamVariati

In [193]:
ising.train(max_epochs= 600)

Epoch 1 TotalParamVariation 1.039811912152953
Epoch 2 TotalParamVariation 1.0399087674052108
Epoch 3 TotalParamVariation 1.040006570628456
Epoch 4 TotalParamVariation 1.040104252054163
Epoch 5 TotalParamVariation 1.0402018118244487
Epoch 6 TotalParamVariation 1.0402992500813
Epoch 7 TotalParamVariation 1.0403965669665733
Epoch 8 TotalParamVariation 1.0404937626219963
Epoch 9 TotalParamVariation 1.0405908371891675
Epoch 10 TotalParamVariation 1.0406877908095546
Epoch 11 TotalParamVariation 1.0407846236244978
Epoch 12 TotalParamVariation 1.0408813357752067
Epoch 13 TotalParamVariation 1.0409779274027626
Epoch 14 TotalParamVariation 1.0410743986481172
Epoch 15 TotalParamVariation 1.0411707496520932
Epoch 16 TotalParamVariation 1.041266980555385
Epoch 17 TotalParamVariation 1.041363091498557
Epoch 18 TotalParamVariation 1.041459082622046
Epoch 19 TotalParamVariation 1.0415549540661595
Epoch 20 TotalParamVariation 1.041650705971076
Epoch 21 TotalParamVariation 1.0417463384768457
Epoch 22 To

Epoch 350 TotalParamVariation 1.1902994002023455
Epoch 351 TotalParamVariation 1.1907099683912197
Epoch 352 TotalParamVariation 1.1911202965127805
Epoch 353 TotalParamVariation 1.1915303847833418
Epoch 354 TotalParamVariation 1.1919402334190152
Epoch 355 TotalParamVariation 1.1923498426357122
Epoch 356 TotalParamVariation 1.1927592126491422
Epoch 357 TotalParamVariation 1.1931683436748148
Epoch 358 TotalParamVariation 1.1935772359280383
Epoch 359 TotalParamVariation 1.1939858896239213
Epoch 360 TotalParamVariation 1.1943943049773713
Epoch 361 TotalParamVariation 1.194802482203096
Epoch 362 TotalParamVariation 1.1952104215156034
Epoch 363 TotalParamVariation 1.1956181231292011
Epoch 364 TotalParamVariation 1.1960255872579972
Epoch 365 TotalParamVariation 1.196432814115901
Epoch 366 TotalParamVariation 1.1968398039166215
Epoch 367 TotalParamVariation 1.1972465568736685
Epoch 368 TotalParamVariation 1.1976530732003536
Epoch 369 TotalParamVariation 1.198059353109789
Epoch 370 TotalParamVar

#### Only H

In [31]:
class SimpleIsingModel:
    def __init__(self, data, lr=0.1):
        self.lr = lr
        
        # Possible values the spins can take
        self.spin_values = [-1,1]
        
        self.data_activations_matrix = pd.DataFrame(activations_list, columns=['sigma_0', 'sigma_1', 'sigma_2', 'sigma_3'])
        # Calculate the true data expectations
        self.data_spins_correlation = self.data_activations_matrix.corr().values
        lind = np.tril_indices_from(self.data_spins_correlation)
        self.data_spins_correlation[lind]=0
        self.data_spins_average = self.data_activations_matrix.mean().values
        
        self.num_spins = self.data_activations_matrix.shape[1]
        self.num_samples_data = self.data_activations_matrix.shape[0]
                
        # Calculate the initial H vector, each entry proportional to its probability (according to Hinton)
        #self.H = self.data_activations_matrix.sum()/self.num_samples_data
        # Try with random H
        self.H = np.random.normal(loc=0.0, scale=.01, size=self.num_spins)
    
    # Calcualte the energy of the model's parameters given a sigma state
    def calculate_energy(self, sigma):
        energy = 0

        for i in range(0, self.num_spins):
            energy += sigma[i]*self.H[i]

        return - energy
    
    # Flip a randomly selected spin
    def flip_spin(self, _sigma):
        sigma = copy.deepcopy(_sigma)
        spin_to_flip = random.choice(range(0, self.num_spins))

        spin = sigma[spin_to_flip]
        if spin == self.spin_values[0]:
            sigma[spin_to_flip] = self.spin_values[1]
        else:
            sigma[spin_to_flip] = self.spin_values[0]
        return sigma
        
        
    
    # Train the exact model
    def train(self, max_epochs = 500):
        
        totalParamVariation = math.inf
        stopCondition = 0.005
        
        epoch = 1
        
        while totalParamVariation > stopCondition and epoch < 500:
            # Calculate P(sigma) for every possible combination in the model

            prob_dict = {}
            for sigma in product(self.spin_values, repeat=self.num_spins):
                prob_dict[sigma] = math.exp(-self.calculate_energy(sigma))

            # Calculate the partition function
            Z = sum(prob_dict.values())

            # Normalize dividing by the partition function
            for sigma in prob_dict.keys():
                prob_dict[sigma] = prob_dict[sigma]/Z

            mean_model_exp = np.zeros(self.H.shape)
            for sigma in product(self.spin_values, repeat=self.num_spins):
                # Calculate the expectation of the averages
                for i in range(0, self.num_spins):
                    mean_model_exp[i] += sigma[i]*prob_dict[sigma]

            # Calculate the step size for every H_{i}
            stepH = self.lr * (self.data_spins_average - mean_model_exp)
            # Calculate the variation of J for the termination condition
            deltaH = abs(self.H - stepH).sum()
            # Take the step
            self.H = self.H + stepH
            totalParamVariation = deltaH
            
            print('Epoch', epoch, 'TotalParamVariation', totalParamVariation)
            
            epoch += 1

        print(self.H)
        print(self.data_spins_correlation)
        
    def random_walk(self, max_steps=40):
        # Generate a random initial spin configuration
        model_activations_list = []
        sigma_0 = random.choices([-1,1], k=self.num_spins)
        
        # Calculate the energy at this time
        energy_0 = self.calculate_energy(sigma_0)
        
        for i in range(0, max_steps):
            
            # Flip a spin and calculate the new energy
            sigma_t = self.flip_spin(sigma_0)
            energy_t = self.calculate_energy(sigma_t)
            delta_energy = energy_t - energy_0

            if delta_energy<=0: # If delta_energy<0, accept the spin flip, energetically favourable
                model_activations_list.append(sigma_t)
            

            else: # Check if transition probability p < uniform r
                p = math.exp(-delta_energy)
                r = random.random()

                if r<=p: # Accept spin flip
                    model_activations_list.append(sigma_t)
                else: # Refuse spin flip
                    model_activations_list.append(sigma_0)
            
            sigma_0 = sigma_t
            energy_0 = energy_t
        
        return model_activations_list
    
    def train_metropolis(self, max_epochs = 300):
        random.seed(3)
        totalParamVariation = math.inf
        stopCondition = 0.0005
        
        epoch = 1
        randomWalkLength = 40
        
        print('Epoch 0')
        print(self.H)
        
        while totalParamVariation > stopCondition and epoch < max_epochs:
            model_activations_list = self.random_walk(randomWalkLength)   
            model_activations_matrix = pd.DataFrame(model_activations_list, columns=['sigma_0', 'sigma_1', 'sigma_2', 'sigma_3'])
            # Calculate teh data expectations
            model_spins_correlation = model_activations_matrix.corr().values
            lind = np.tril_indices_from(model_spins_correlation)
            model_spins_correlation[lind]=0
            model_spins_average = model_activations_matrix.mean().values
            
            # Calculate the step size for every H_{i}
            stepH = self.lr * (self.data_spins_average - model_spins_average)
            # Calculate the variation of J for the termination condition
            deltaH = abs(self.H - stepH).sum()
            # Take the step
            self.H = self.H + stepH
            totalParamVariation = deltaH
            
            print('Epoch', epoch, 'TotalParamVariation', totalParamVariation)
            print(stepH, 'Step')
            print(self.H, 'H')

            
            epoch +=1

        #print(self.H)
        #print(self.data_spins_correlation)
        
        print(self.data_spins_average, 'Spins average')


In [37]:
activations_list = [[1,-1,1,-1], [1,-1,1,-1], [-1,1,-1,-1], [-1,1,1,1], [1,1,1,-1], [-1,-1,1,1], [1,-1,-1,1], [1,-1,1,1]]
s_ising = SimpleIsingModel(activations_list, lr=0.0005)


In [38]:
s_ising.train_metropolis(max_epochs = 1000)

Epoch 0
[-0.00581437  0.0156781   0.00527893 -0.00328347]
Epoch 1 TotalParamVariation 0.030054861255895793
[2.0e-04 1.5e-04 1.0e-04 5.0e-05] Step
[-0.00561437  0.0158281   0.00537893 -0.00323347] H
Epoch 2 TotalParamVariation 0.030479861255895795
[ 3.25e-04 -7.50e-05  2.00e-04  2.25e-04] Step
[-0.00528937  0.0157531   0.00557893 -0.00300847] H
Epoch 3 TotalParamVariation 0.0294798612558958
[ 2.50e-05 -1.25e-04  0.00e+00 -3.00e-04] Step
[-0.00526437  0.0156281   0.00557893 -0.00330847] H
Epoch 4 TotalParamVariation 0.0301298612558958
[ 7.5e-05 -3.5e-04  1.0e-04  2.5e-05] Step
[-0.00518937  0.0152781   0.00567893 -0.00328347] H
Epoch 5 TotalParamVariation 0.029179861255895802
[ 1.25e-04  7.50e-05  1.75e-04 -1.25e-04] Step
[-0.00506437  0.0153531   0.00585393 -0.00340847] H
Epoch 6 TotalParamVariation 0.030079861255895797
[ 3.25e-04 -2.50e-05  1.00e-04  1.50e-04] Step
[-0.00473937  0.0153281   0.00595393 -0.00325847] H
Epoch 7 TotalParamVariation 0.0290798612558958
[-1.75e-04 -1.25e-04  5

Epoch 82 TotalParamVariation 0.03765112365218794
[ 0.000125 -0.000175  0.00015   0.      ] Step
[ 0.00598563  0.0047531   0.02460393 -0.00250847] H
Epoch 83 TotalParamVariation 0.03790112365218794
[ 7.50e-05 -1.25e-04  2.00e-04  2.00e-04] Step
[ 0.00606063  0.0046281   0.02480393 -0.00230847] H
Epoch 84 TotalParamVariation 0.03772612365218794
[ 0.00025  -0.000175  0.0002    0.0002  ] Step
[ 0.00631063  0.0044531   0.02500393 -0.00210847] H
Epoch 85 TotalParamVariation 0.03757612365218794
[ 1.25e-04 -5.00e-05 -7.50e-05 -3.00e-04] Step
[ 0.00643563  0.0044031   0.02492893 -0.00240847] H
Epoch 86 TotalParamVariation 0.03805112365218795
[ 0.000125 -0.000125  0.00025   0.000125] Step
[ 0.00656063  0.0042781   0.02517893 -0.00228347] H
Epoch 87 TotalParamVariation 0.03815112365218794
[ 5.00e-05 -5.00e-05  2.75e-04  1.25e-04] Step
[ 0.00661063  0.0042281   0.02545393 -0.00215847] H
Epoch 88 TotalParamVariation 0.03805112365218794
[ 2.25e-04 -2.50e-05  3.00e-04  1.00e-04] Step
[ 0.00683563  0.

[ 5.0e-05  1.0e-04  1.5e-04 -2.0e-04] Step
[ 0.01613563 -0.0043719   0.04305393 -0.00013347] H
Epoch 173 TotalParamVariation 0.06329493289099931
[ 5.00e-05 -7.50e-05  2.75e-04  0.00e+00] Step
[ 0.01618563 -0.0044469   0.04332893 -0.00013347] H
Epoch 174 TotalParamVariation 0.06366993289099933
[-5.00e-05 -1.75e-04  4.00e-04  1.00e-04] Step
[ 1.61356312e-02 -4.62190462e-03  4.37289290e-02 -3.34681019e-05] H
Epoch 175 TotalParamVariation 0.0641199328909993
[ 5.00e-05 -4.00e-04  7.50e-05  1.25e-04] Step
[ 1.61856312e-02 -5.02190462e-03  4.38039290e-02  9.15318981e-05] H
Epoch 176 TotalParamVariation 0.06495299668715643
[ 2.50e-05  7.50e-05  2.25e-04 -2.50e-05] Step
[ 1.62106312e-02 -4.94690462e-03  4.40289290e-02  6.65318981e-05] H
Epoch 177 TotalParamVariation 0.06496993289099931
[ 0.0001   -0.000175  0.00015   0.000275] Step
[ 0.01631063 -0.0051219   0.04417893  0.00034153] H
Epoch 178 TotalParamVariation 0.06602799668715643
[ 7.5e-05 -5.0e-05  0.0e+00 -2.0e-04] Step
[ 0.01638563 -0.0051

Epoch 265 TotalParamVariation 0.10686993289099933
[7.50e-05 0.00e+00 3.75e-04 2.25e-04] Step
[ 2.57356312e-02 -1.63719046e-02  6.53039290e-02  9.15318981e-05] H
Epoch 266 TotalParamVariation 0.10675299668715643
[ 3.25e-04 -4.50e-04  2.50e-05 -5.00e-05] Step
[ 2.60606312e-02 -1.68219046e-02  6.53289290e-02  4.15318981e-05] H
Epoch 267 TotalParamVariation 0.10756993289099931
[ 0.00015 -0.0002   0.00035  0.0001 ] Step
[ 0.02621063 -0.0170219   0.06567893  0.00014153] H
Epoch 268 TotalParamVariation 0.10897799668715644
[ 1.75e-04  2.50e-05  2.00e-04 -2.75e-04] Step
[ 0.02638563 -0.0169969   0.06587893 -0.00013347] H
Epoch 269 TotalParamVariation 0.10879493289099933
[ 2.75e-04 -7.50e-05  2.50e-04  0.00e+00] Step
[ 0.02666063 -0.0170719   0.06612893 -0.00013347] H
Epoch 270 TotalParamVariation 0.10960299668715645
[ 2.50e-05 -1.50e-04  1.25e-04 -1.75e-04] Step
[ 0.02668563 -0.0172219   0.06625393 -0.00030847] H
Epoch 271 TotalParamVariation 0.11014493289099932
[2.75e-04 1.00e-04 2.00e-04 5.00

Epoch 349 TotalParamVariation 0.1462779966871565
[ 5.00e-05 -7.50e-05  3.75e-04  3.25e-04] Step
[ 0.03616063 -0.0277219   0.08337893  0.00066653] H
Epoch 350 TotalParamVariation 0.14777799668715647
[ 0.0e+00 -2.5e-05  1.5e-04 -2.5e-05] Step
[ 0.03616063 -0.0277469   0.08352893  0.00064153] H
Epoch 351 TotalParamVariation 0.14790299668715648
[-7.50e-05 -2.50e-05  1.75e-04  5.00e-05] Step
[ 0.03608563 -0.0277719   0.08370393  0.00069153] H
Epoch 352 TotalParamVariation 0.14740299668715648
[ 0.000275 -0.000125  0.00025   0.0002  ] Step
[ 0.03636063 -0.0278969   0.08395393  0.00089153] H
Epoch 353 TotalParamVariation 0.14860299668715646
[ 0.0002 -0.0002  0.0002 -0.0001] Step
[ 0.03656063 -0.0280969   0.08415393  0.00079153] H
Epoch 354 TotalParamVariation 0.14922799668715647
[ 0.0002   -0.00015   0.000125 -0.0001  ] Step
[ 0.03676063 -0.0282469   0.08427893  0.00069153] H
Epoch 355 TotalParamVariation 0.14952799668715644
[ 0.0001  -0.00015  0.0002   0.     ] Step
[ 0.03686063 -0.0283969   

...

Epoch 594 TotalParamVariation 0.25727799668715656
[ 2.75e-04 -2.50e-04  2.50e-04 -5.00e-05] Step
[ 0.06363563 -0.0560469   0.13830393  0.00074153] H
Epoch 595 TotalParamVariation 0.25845299668715654
[ 0.00e+00 -1.75e-04  2.50e-05  7.50e-05] Step
[ 0.06363563 -0.0562219   0.13832893  0.00081653] H
Epoch 596 TotalParamVariation 0.25832799668715656
[ 1.0e-04 -2.5e-05  4.0e-04  1.5e-04] Step
[ 0.06373563 -0.0562469   0.13872893  0.00096653] H
Epoch 597 TotalParamVariation 0.2588529966871566
[ 0.000125 -0.0002    0.0002    0.0003  ] Step
[ 0.06386063 -0.0564469   0.13892893  0.00126653] H
Epoch 598 TotalParamVariation 0.2606029966871566
[ 1.0e-04 -2.5e-05  2.5e-05 -2.5e-04] Step
[ 0.06396063 -0.0564719   0.13895393  0.00101653] H
Epoch 599 TotalParamVariation 0.25982799668715656
[ 2.00e-04 -7.50e-05  1.25e-04  1.75e-04] Step
[ 0.06416063 -0.0565469   0.13907893  0.00119153] H
Epoch 600 TotalParamVariation 0.2606779966871565
[ 7.50e-05 -1.25e-04 -2.50e-05  1.25e-04] Step
[ 0.06423563 -0.0566

Epoch 674 TotalParamVariation 0.2953949328909995
[ 1.00e-04 -1.25e-04  2.00e-04 -7.50e-05] Step
[ 0.07321063 -0.0669469   0.15572893 -0.00050847] H
Epoch 675 TotalParamVariation 0.2959949328909996
[ 7.50e-05  7.50e-05  3.25e-04 -7.50e-05] Step
[ 0.07328563 -0.0668719   0.15605393 -0.00058347] H
Epoch 676 TotalParamVariation 0.29681993289099956
[-5.00e-05 -1.50e-04  0.00e+00  1.25e-04] Step
[ 0.07323563 -0.0670219   0.15605393 -0.00045847] H
Epoch 677 TotalParamVariation 0.29644493289099955
[ 5.00e-05 -1.50e-04  0.00e+00 -1.25e-04] Step
[ 0.07328563 -0.0671719   0.15605393 -0.00058347] H
Epoch 678 TotalParamVariation 0.2964699328909996
[ 1.50e-04 -3.25e-04  2.50e-05 -1.25e-04] Step
[ 0.07343563 -0.0674969   0.15607893 -0.00070847] H
Epoch 679 TotalParamVariation 0.2970199328909996
[ 2.5e-05 -3.0e-04  3.0e-04 -7.5e-05] Step
[ 0.07346063 -0.0677969   0.15637893 -0.00078347] H
Epoch 680 TotalParamVariation 0.2976199328909996
[ 0.000175 -0.0002    0.0003   -0.000125] Step
[ 0.07363563 -0.06

Epoch 764 TotalParamVariation 0.33419493289099944
[-2.25e-04  5.00e-05  2.25e-04  2.25e-04] Step
[ 8.23356312e-02 -7.59469046e-02  1.75378929e-01  1.65318981e-05] H
Epoch 765 TotalParamVariation 0.3334449328909994
[2.75e-04 2.50e-05 2.50e-05 7.50e-05] Step
[ 8.26106312e-02 -7.59219046e-02  1.75403929e-01  9.15318981e-05] H
Epoch 766 TotalParamVariation 0.3335279966871565
[ 2.0e-04 -7.5e-05  1.5e-04  7.5e-05] Step
[ 8.28106312e-02 -7.59969046e-02  1.75553929e-01  1.66531898e-04] H
Epoch 767 TotalParamVariation 0.3342279966871566
[ 2.00e-04 -2.50e-05  1.25e-04 -5.00e-05] Step
[ 8.30106312e-02 -7.60219046e-02  1.75678929e-01  1.16531898e-04] H
Epoch 768 TotalParamVariation 0.3345699328909994
[-2.5e-05  0.0e+00  2.5e-04  2.0e-04] Step
[ 0.08298563 -0.0760219   0.17592893  0.00031653] H
Epoch 769 TotalParamVariation 0.3347529966871565
[ 2.00e-04 -7.50e-05  3.25e-04 -1.00e-04] Step
[ 0.08318563 -0.0760969   0.17625393  0.00021653] H
Epoch 770 TotalParamVariation 0.3353029966871566
[ 1.5e-04 

Epoch 849 TotalParamVariation 0.36781993289099935
[ 2.00e-04 -2.00e-04 -7.50e-05  1.75e-04] Step
[ 0.08983563 -0.0839719   0.19370393 -0.00060847] H
Epoch 850 TotalParamVariation 0.3683199328909994
[-1.00e-04  1.25e-04  0.00e+00 -2.50e-05] Step
[ 0.08973563 -0.0838469   0.19370393 -0.00063347] H
Epoch 851 TotalParamVariation 0.3674449328909994
[-0.000175 -0.00015   0.000325 -0.000175] Step
[ 0.08956063 -0.0839969   0.19402893 -0.00080847] H
Epoch 852 TotalParamVariation 0.36789493289099945
[1.50e-04 2.50e-05 5.00e-04 1.25e-04] Step
[ 0.08971063 -0.0839719   0.19452893 -0.00068347] H
Epoch 853 TotalParamVariation 0.3686699328909994
[ 2.5e-04 -5.0e-05 -5.0e-05  2.5e-05] Step
[ 0.08996063 -0.0840219   0.19447893 -0.00065847] H
Epoch 854 TotalParamVariation 0.36856993289099943
[ 2.75e-04 -5.00e-05  1.25e-04 -1.00e-04] Step
[ 0.09023563 -0.0840719   0.19460393 -0.00075847] H
Epoch 855 TotalParamVariation 0.36911993289099937
[-2.50e-05 -1.50e-04  4.75e-04  5.00e-05] Step
[ 0.09021063 -0.0842

Epoch 936 TotalParamVariation 0.39904493289099935
[ 5.00e-05 -4.75e-04  2.25e-04  0.00e+00] Step
[ 9.72106312e-02 -9.18969046e-02  2.11353929e-01 -8.34681019e-05] H
Epoch 937 TotalParamVariation 0.4002949328909993
[ 1.50e-04 -5.00e-05  1.75e-04  1.25e-04] Step
[ 9.73606312e-02 -9.19469046e-02  2.11528929e-01  4.15318981e-05] H
Epoch 938 TotalParamVariation 0.4007699328909993
[-2.5e-05 -1.0e-04  2.5e-05  7.5e-05] Step
[ 9.73356312e-02 -9.20469046e-02  2.11553929e-01  1.16531898e-04] H
Epoch 939 TotalParamVariation 0.40130299668715647
[-1.25e-04 -2.50e-05  0.00e+00 -1.50e-04] Step
[ 9.72106312e-02 -9.20719046e-02  2.11553929e-01 -3.34681019e-05] H
Epoch 940 TotalParamVariation 0.4002199328909993
[ 0.00025 -0.0002   0.0003   0.0001 ] Step
[ 9.74606312e-02 -9.22719046e-02  2.11853929e-01  6.65318981e-05] H
Epoch 941 TotalParamVariation 0.40182799668715646
[ 0.000125  0.000125  0.       -0.000175] Step
[ 9.75856312e-02 -9.21469046e-02  2.11853929e-01 -1.08468102e-04] H
Epoch 942 TotalParamV